# Download weather data for Florence

or any other city

In [ ]:
from datetime import datetime

import polars as pl
from meteostat import Hourly, Stations

In [ ]:
CITY = "Florence"
COUNTRY = "Italy"

In [ ]:
# Load our cities dataset and find the city coordinates
cities = pl.read_parquet("../data/worldcities.parquet")
city_info = cities.filter(city=CITY, country=COUNTRY).row(0, named=True)
city_info


In [ ]:
# Find nearby weather stations
s = Stations()
s = s.nearby(
    lat=city_info["lat"],
    lon=city_info["lng"],
    radius=100000  # in m?
)
stations = s.fetch(10)
stations   # pandas DataFrame

In [ ]:
# Take the first one
station_id = stations.index[0]
station_id

In [ ]:
# Get all data
source = Hourly(station_id, start=datetime(1932, 1, 1), end=datetime(2024, 5, 16))
source

In [ ]:
# Create a DataFrame from it
df = pl.from_pandas(source.fetch().reset_index(), nan_to_null=True)
# Be explicit about the Nones
df = df.fill_nan(None)
df

In [ ]:
# Save
df.write_parquet(f"../data/{CITY.lower()}-meteostat.parquet")